In [1]:
import numpy as np
import xarray as xr
import pandas as pd
from skimage import transform
import torch

from universal_computation.fpt import FPT

In [2]:
if torch.cuda.is_available():
    device = 'cuda:0'
else:
    device = 'cpu'
print(device)

cuda:0


In [3]:
model = FPT(input_dim=405, output_dim=16000)
model.to(device)

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


FPT(
  (sequence_model): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((76

In [4]:
lats = np.array([30, 50])
lons = np.array([-125, -75])

In [6]:
target_date = '2005-03-10'

# 2/29 on leap years need to be removed because
# cm5a and cm3 data doesn't record those dates
# even though CHIRPS and cgcm3 do
leap_year_dates = np.array([np.datetime64(f'{year}-02-29')
                            for year in range(1984, 2005, 4)])

# convert date ranges because not all of these datasets
# are operating on the same precision which messes with
# indexing
def fix_dates(xdata):
    xdata['time'] = pd.date_range(
        xdata['time'][0].values,
        periods=len(xdata['time']),
        freq='1D').floor('D')
    return xdata

output = []
# don't include CCSM4 since the years seem wrong for it
input_dirs = ['IPSL-CM5A-LR', 'GFDL-CM3', 'MRI-CGCM3']
input_shape = (18, 45)  # shape of the largest input (CM5A)
for direc in input_dirs:
    ds = xr.open_mfdataset(f'/u/scratch/j/jkjewik/fpt4sd/CMIP5/{direc}/*.nc')
    xdata = ds.prc.sel(
        time=target_date,
        lat=slice(*lats),
        lon=slice(*lons%360))  # convert 360deg to 180deg system
    if direc == 'MRI-CGCM3':
        xdata = fix_dates(xdata)
        mask = ~xdata['time'].isin(leap_year_dates)
    else:
        mask = ~xdata.isin(leap_year_dates)
    xdata = xdata.where(mask, drop=True)
    npdata = xdata.values
    npdata = np.moveaxis(npdata, 0, -1)  # lat x lon x time
    npdata = transform.resize(npdata, input_shape)
    npdata *= 86400  # convert kg,m-2,s-1 to mm,day-1
    npdata = np.moveaxis(npdata, -1, 0)  # time x lat x lon
    output.append(npdata)
output = np.stack(output, 1)  # time x source x lat x lon

NotImplementedError: Don't yet support nd fancy indexing